In [1]:
from bs4 import BeautifulSoup
from requests import get

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math 
import os

In [2]:
# Function for remove comma within numbers
def removeCommas(string): 
    string = string.replace(',','')
    return string 

# Scrap data from worldmeter

In [155]:
# Test if we can scrap info from worldometers
# The communication with website is ok if the response is 200
headers = ({'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
worldometers = "https://www.worldometers.info/coronavirus/#countries"
response = get(worldometers, headers=headers)
response

<Response [200]>

In [156]:
# Scrap all content from the website
html_soup = BeautifulSoup(response.text, 'html.parser')
# After inspect the website content, data are stored inside tag 'tbody' and table header is 'thead'
table_contents = html_soup.find_all('tbody')
table_header = html_soup.find_all('thead')

# Header for the table
header = []
for head_title in table_header[0].find_all('th'):    
    header.append(str(head_title.contents))

# Save value into columns
CountryName = []
TotalCases = []
NewCases = []
TotalDeaths = []
NewDeaths = []
TotalRecovered = []
ActiveCases = []
SeriousCritical = []

for row in table_contents[0].find_all('tr'):
    cells = row.find_all('td')
    if len(cells[0].find_all('a')) >= 1:
        CountryName.append(cells[0].find_all('a')[0].contents[0])
    elif len(cells[0].find_all('span')) >= 1:
        CountryName.append(cells[0].find_all('span')[0].contents[0])   
    else:
        CountryName.append(cells[0].contents[0])
    
    
    if len(cells[1].contents) >=1:
        TotalCases.append(cells[1].contents[0])
    else:
        TotalCases.append(0)
    
    if len(cells[2].contents) >= 1:
        NewCases.append(cells[2].contents[0])
    else:
        NewCases.append(0)
        
    
    if len(cells[3].contents) >= 1:
        TotalDeaths.append(cells[3].contents[0])
    else:
        TotalDeaths.append(0)

    
    if len(cells[4].contents) >= 1:
        NewDeaths.append(cells[4].contents[0])
    else:
        NewDeaths.append(0)
    
    if len(cells[5].contents) >= 1:
        TotalRecovered.append(cells[5].contents[0])
    else:
        TotalRecovered.append(0)
        
    if len(cells[6].contents) >= 1:
        ActiveCases.append(cells[6].contents[0])
    else:
        ActiveCases.append(0)
    
    if len(cells[7].contents) >= 1:
        SeriousCritical.append(cells[7].contents[0])
    else:
        SeriousCritical.append(0)
        
        
CaseTable = pd.DataFrame({header[0]: CountryName,
                          header[1]: TotalCases,
                          header[2]: NewCases,
                          header[3]: TotalDeaths,
                          header[4]: NewDeaths,                          
                          header[5]: TotalRecovered,
                          header[6]: ActiveCases,
                          header[7]: SeriousCritical,
                          })  

CaseTable.head(40)

,"['Country,', <br>Other</br>]","['Total', <br>Cases</br>]","['New', <br>Cases</br>]","['Total', <br/>, 'Deaths']","['New', <br/>, 'Deaths']","['Total', <br/>, 'Recovered']","['Active', <br/>, 'Cases']","['Serious,', <br/>, 'Critical']"
0,China,"81,171",+78,"3,277",+7,"73,159","4,735","1,573"
1,Italy,"63,927",0,"6,077",0,"7,432","50,418","3,204"
2,USA,"46,089","+2,355",579,+26,295,"45,215","1,040"
3,Spain,"35,136",0,"2,311",0,"3,355","29,470","2,355"
4,Germany,"29,056",0,123,0,453,"28,480",23
5,Iran,"23,049",0,"1,812",0,"8,376","12,861",0
6,France,"19,856",0,860,0,"2,200","16,796","2,082"
7,S. Korea,"9,037",+76,120,+9,"3,507","5,410",59
8,Switzerland,"8,795",0,120,0,131,"8,544",141
9,UK,"6,650",0,335,0,135,"6,180",20


In [157]:
CaseTable.tail(40)

,"['Country,', <br>Other</br>]","['Total', <br>Cases</br>]","['New', <br>Cases</br>]","['Total', <br/>, 'Deaths']","['New', <br/>, 'Deaths']","['Total', <br/>, 'Recovered']","['Active', <br/>, 'Cases']","['Serious,', <br/>, 'Critical']"
156,Bahamas,4,0,,0,0,4,0
157,Congo,4,0,,0,0,4,0
158,Greenland,4,0,,0,0,4,0
159,Guinea,4,0,,0,0,4,0
160,Namibia,4,0,,0,0,4,0
161,Eswatini,4,0,,0,0,4,0
162,Zimbabwe,3,0,1,0,0,2,0
163,Angola,3,0,,0,0,3,0
164,Antigua and Barbuda,3,0,,0,0,3,0
165,Cabo Verde,3,0,,0,0,3,0


In [158]:
caseTableSimple = CaseTable[[CaseTable.columns[0], CaseTable.columns[1], CaseTable.columns[3], CaseTable.columns[5]]]
caseTableSimple.columns = ['Country/Region', 'Confirmed', 'Deaths', 'Recovered']
# Set data type as string first for manuipulation
caseTableSimple = caseTableSimple.astype({'Country/Region':str,'Confirmed':str,'Deaths':str, 'Recovered':str})
# Remove the last row of total number (changed on 20200310, worldmeter moved this row as next tbody)
#caseTableSimple = caseTableSimple.iloc[:-1,:]
# Remove lead and tail space for each element
caseTableSimple = caseTableSimple.apply(lambda x: x.str.strip())
# Remove comma for each element
caseTableSimple = caseTableSimple.applymap(removeCommas)
# Replace empty str with zero. This include row of 'Diamond Princess' (its name is empty)
caseTableSimple = caseTableSimple.replace('', '0')
# After string manipulation, convert data type as correct type
caseTableSimple = caseTableSimple.astype({'Country/Region':'str',
                                          'Confirmed':'int',
                                          'Deaths':'int',
                                          'Recovered':'int',                                          
                                         })
# Data for these countries come from other source
removeRegion = ['China', 'Canada', 'Australia', 'USA']
for i in removeRegion:
    caseTableSimple.drop(caseTableSimple[caseTableSimple['Country/Region'] == i].index, axis=0, inplace=True)

# Change Country name the same as my old data 
if 'S. Korea' in list(caseTableSimple['Country/Region']):
    caseTableSimple = caseTableSimple.replace('S. Korea', 'South Korea')

# Add column 'Province/State' with empty value
caseTableSimple['Province/State'] =''

# In my old data, 'Diamond Princess' is represented by 'Yokohama' in the column of 'Province/State'
if 'Diamond Princess' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Diamond Princess',].index, 'Province/State'] = 'Yokohama'
    caseTableSimple['Country/Region'].replace({'Diamond Princess':'Japan'}, inplace=True)

# In my old data, 'Belgium' has 'Brussels' in the column of 'Province/State'
if 'Belgium' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Belgium',].index, 'Province/State'] = 'Brussels'

# In my old data, I used 'Macau' not 'Macao'
if 'Macao' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Macao',].index, 'Province/State'] = 'Macau'
    caseTableSimple['Country/Region'].replace({'Macao':'Macau'}, inplace=True)

# In my old data, 'Hong Kong' has 'Hong Kong' in the column of 'Province/State'
if 'Hong Kong' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Hong Kong',].index, 'Province/State'] = 'Hong Kong'

# In my old data, 'Taiwan' has 'Taiwan' in the column of 'Province/State'
if 'Taiwan' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Taiwan',].index, 'Province/State'] = 'Taiwan'

# In my old data, I used 'United Arab Emirates' not 'UAE'
if 'UAE' in list(caseTableSimple['Country/Region']):
    caseTableSimple['Country/Region'].replace({'UAE':'United Arab Emirates'}, inplace=True)

if 'Réunion' in list(caseTableSimple['Country/Region']):
    caseTableSimple['Country/Region'].replace({'Réunion':'Reunion'}, inplace=True)
    
if 'Curaçao' in list(caseTableSimple['Country/Region']):
    caseTableSimple['Country/Region'].replace({'Curaçao':'Curacao'}, inplace=True)

# In my old data I used US time as Last Update time
currentTime = datetime.now()
lastUpdateTime = currentTime.strftime('%m/%d/%Y %H:%M')
# Remove the first number (This only works for month number less than 10)
lastUpdateTime[1:]
caseTableSimple['Last Update'] = lastUpdateTime[1:]

# Reorder list as all old data
columnList = caseTableSimple.columns.tolist()
columnList =[columnList[i] for i in [4, 0, 5, 1, 2, 3]]
caseTableSimple = caseTableSimple[columnList]

In [159]:
caseTableSimple.tail(20)

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
176,,Nepal,3/24/2020 14:08,2,0,1
177,,Bhutan,3/24/2020 14:08,2,0,0
178,,Chad,3/24/2020 14:08,2,0,0
179,,Dominica,3/24/2020 14:08,2,0,0
180,,Mauritania,3/24/2020 14:08,2,0,0
181,,Myanmar,3/24/2020 14:08,2,0,0
182,,Nicaragua,3/24/2020 14:08,2,0,0
183,,Sint Maarten,3/24/2020 14:08,2,0,0
184,,Belize,3/24/2020 14:08,1,0,0
185,,Eritrea,3/24/2020 14:08,1,0,0


# Scrap data for US_CAN

In [160]:
# Test if we can scrap info from worldometers
# The communication with website is ok if the response is 200
US_Canada = "https://coronavirus.1point3acres.com/zh"
response2 = get(US_Canada, headers=headers)
response2

<Response [200]>

In [161]:
# Scrap all content from the website
html_soup2 = BeautifulSoup(response2.text, 'html.parser')

In [162]:
# Since they change class index everyday, this code is for finding the new index.
indexList = []
for span in html_soup2.find_all('span'):
    # Only retain 'span' that has contents
    if len(span.contents):
        # Since we only need to find index for table, use one of the table head as target word to locate index
        if span.contents[0] == 'Location':
            # Store the index inside a list
            indexList.append(span['class'][0])

In [163]:
# The first index is for US table and the 2nd index is for Canada table. Do not casr about the rest inside the list.
USindex, CANindex, _ = indexList

In [164]:
# Check if the index return right data
html_soup2.find_all('span', class_=USindex)

[<span class="jsx-314244412" style="color:#ee7500">🎉 Click here to check the state testing data/location</span>,
 <span class="jsx-314244412">Location</span>,
 <span class="jsx-314244412">Confirmed</span>,
 <span class="jsx-314244412">Deaths</span>,
 <span class="jsx-314244412">Fatality rate</span>,
 <span class="jsx-314244412">Source</span>,
 <span class="jsx-314244412">United States</span>,
 <span class="jsx-314244412">46363</span>,
 <span class="jsx-314244412">585</span>,
 <span class="jsx-314244412">1.3%</span>,
 <span class="jsx-314244412"><a class="jsx-314244412" href="https://www.cdc.gov" target="_blank">CDC</a></span>,
 <span class="jsx-314244412">New York<i aria-label="icon: warning" class="anticon anticon-warning emergency-icon" tabindex="-1"><svg aria-hidden="true" class="" data-icon="warning" fill="currentColor" focusable="false" height="1em" viewbox="64 64 896 896" width="1em"><path d="M464 720a48 48 0 1 0 96 0 48 48 0 1 0-96 0zm16-304v184c0 4.4 3.6 8 8 8h48c4.4 0 8-3.6 8-

In [165]:
Locations = []
Confirmed = []
Recovered = []
Deaths = []
list1 = range(1, len(html_soup2.find_all('span', class_=USindex))-4, 5)
list2 = range(2, len(html_soup2.find_all('span', class_=USindex))-3, 5)
list3 = range(3, len(html_soup2.find_all('span', class_=USindex))-2, 5)
list4 = range(4, len(html_soup2.find_all('span', class_=USindex))-1, 5)

for index in list1:
    if len(html_soup2.find_all('span', class_=USindex)[index].contents):
        Locations.append(html_soup2.find_all('span', class_=USindex)[index].contents[0])
    else:
        Locations.append(0)
for index in list2:
    if len(html_soup2.find_all('span', class_=USindex)[index].contents):
        try:
            Confirmed.append(html_soup2.find_all('span', class_=USindex)[index].contents[1])
        except:
            Confirmed.append(html_soup2.find_all('span', class_=USindex)[index].contents[0])
    else:
        Confirmed.append(0)
for index in list3:
    # They do not provide recovered cases number anymore.
    #if len(html_soup2.find_all('span', class_=USindex)[index].contents):
    #    Recovered.append(html_soup2.find_all('span', class_=USindex)[index].contents[0])
    #else:
    Recovered.append(0)
for index in list3:
    if len(html_soup2.find_all('span', class_=USindex)[index].contents):
        try:
            Deaths.append(html_soup2.find_all('span', class_=USindex)[index].contents[1])
        except:
            Deaths.append(html_soup2.find_all('span', class_=USindex)[index].contents[0])
    else:
        Deaths.append(0)
    
US_data = pd.DataFrame({'Province/State':Locations,
                        'Confirmed':Confirmed,
                        'Deaths':Deaths,
                        #'Recovered':Recovered,  
                            })

# Remove rows that are not data
US_data.drop(US_data[US_data['Deaths'] == 'Deaths'].index, axis=0, inplace=True)

# Remove rows that are not data
US_data.drop(US_data[US_data['Province/State'] == 'United States'].index, axis=0, inplace=True)

# Replace Washington, D.C. as Washington DC
if 'Washington, D.C.' in list(US_data['Province/State']):
    US_data['Province/State'].replace({'Washington, D.C.':'Washington DC'}, inplace=True)

# Replace Washington as WA
if 'Washington' in list(US_data['Province/State']):
    US_data['Province/State'].replace({'Washington':'WA'}, inplace=True)
    
# Replace Grand Princess as From Grand Princess
if 'Grand Princess' in list(US_data['Province/State']):
    US_data['Province/State'].replace({'Grand Princess':'From Grand Princess'}, inplace=True)
    
# Replace Diamond Princess as From Diamond Princess cruise
if 'Diamond Princess' in list(US_data['Province/State']):
    US_data['Province/State'].replace({'Diamond Princess':'From Diamond Princess cruise'}, inplace=True)
    
# Assign 0 in column Province/State as unassigned
if 0 in list(US_data['Province/State']):
    US_data.at[US_data.loc[US_data['Province/State'] == 0,].index, 'Province/State'] = 'Unassigned'

In [166]:
US_data

,Province/State,Confirmed,Deaths
2,New York,23230,183
3,New Jersey,2844,27
4,WA,2221,111
5,California,2212,43
6,Michigan,1329,15
7,Illinois,1285,12
8,Florida,1227,18
9,Louisiana,1172,35
10,Georgia,803,26
11,Massachusetts,777,9


In [167]:
Locations = []
Confirmed = []
Recovered = []
Deaths = []
list1 = range(0, len(html_soup2.find_all('span', class_=CANindex))-4, 5)
list2 = range(1, len(html_soup2.find_all('span', class_=CANindex))-3, 5)
list3 = range(2, len(html_soup2.find_all('span', class_=CANindex))-2, 5)
list4 = range(3, len(html_soup2.find_all('span', class_=CANindex))-1, 5)

for index in list1:
    if len(html_soup2.find_all('span', class_=CANindex)[index].contents):
        Locations.append(html_soup2.find_all('span', class_=CANindex)[index].contents[0])
    else:
        Locations.append(0)
for index in list2:
    if len(html_soup2.find_all('span', class_=CANindex)[index].contents):
        try:
            Confirmed.append(html_soup2.find_all('span', class_=CANindex)[index].contents[1])
        except:
            Confirmed.append(html_soup2.find_all('span', class_=CANindex)[index].contents[0])
    else:
        Confirmed.append(0)
for index in list3:
    #. They do not provide recovered cases number
    #if len(html_soup2.find_all('span', class_=CANindex)[index].contents):
    #    Recovered.append(html_soup2.find_all('span', class_=CANindex)[index].contents[0])
    #else:
    Recovered.append(0)
for index in list3:
    if len(html_soup2.find_all('span', class_=CANindex)[index].contents):
        try:
            Deaths.append(html_soup2.find_all('span', class_=CANindex)[index].contents[1])
        except:
            Deaths.append(html_soup2.find_all('span', class_=CANindex)[index].contents[0])
    else:
        Deaths.append(0)
    
CAN_data = pd.DataFrame({'Province/State':Locations,
                         'Confirmed':Confirmed,
                         'Deaths':Deaths,
                         #'Recovered':Recovered,  
                            })

# Remove rows that are not data
CAN_data.drop(CAN_data[CAN_data['Deaths'] == 'Deaths'].index, axis=0, inplace=True)

# Remove rows that are not data
CAN_data.drop(CAN_data[CAN_data['Province/State'] == 'Canada'].index, axis=0, inplace=True)

In [168]:
CAN_data

,Province/State,Confirmed,Deaths
2,Quebec,628,4
3,Ontario,503,6
4,British Columbia,472,13
5,Alberta,301,1
6,Saskatchewan,66,0
7,Nova Scotia,41,0
8,Newfoundland and Labrador,24,0
9,Manitoba,20,0
10,New Brunswick,17,0
11,Grand Princess,13,0


In [169]:
US_Can_data = pd.concat([US_data, CAN_data], ignore_index=True)
US_Can_data = US_Can_data.apply(lambda x: x.str.strip())
US_Can_data

,Province/State,Confirmed,Deaths
0,New York,23230,183
1,New Jersey,2844,27
2,WA,2221,111
3,California,2212,43
4,Michigan,1329,15
...,...,...,...
66,Grand Princess,13,0
67,Prince Edward Island,3,0
68,Yukon,2,0
69,Northwest Territories,1,0


In [170]:
nameList = pd.read_csv('./web_data/statesNameTranslation.csv')

In [171]:
US_Can_data_EN = pd.merge(US_Can_data, nameList, how = 'left', left_on = 'Province/State', right_on = 'English')
US_Can_data_EN = US_Can_data_EN.drop(['Chinese', 'Province/State', 'Abbr.'], axis=1)
US_Can_data_EN['Last Update'] = lastUpdateTime[1:]
US_Can_data_EN.rename(columns={'English':'Province/State'}, inplace=True)
US_Can_data_EN = US_Can_data_EN.drop(US_Can_data_EN[US_Can_data_EN['Province/State'] == 'Wuhan Evacuee'].index, axis=0)
columnOrder = ['Province/State', 'Country/Region', 'Last Update','Confirmed', 'Deaths', 'Recovered']

US_Can_data_EN = US_Can_data_EN[columnOrder]
US_Can_data_EN 

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,New York,US,3/24/2020 14:08,23230,183,0.0
1,New Jersey,US,3/24/2020 14:08,2844,27,1.0
2,WA,US,3/24/2020 14:08,2221,111,1.0
3,California,US,3/24/2020 14:08,2212,43,6.0
4,Michigan,US,3/24/2020 14:08,1329,15,0.0
...,...,...,...,...,...,...
66,NaN,NaN,3/24/2020 14:08,13,0,NaN
67,Prince Edward Island,Canada,3/24/2020 14:08,3,0,0.0
68,Yukon,Canada,3/24/2020 14:08,2,0,0.0
69,Northwest Territories,Canada,3/24/2020 14:08,1,0,0.0


In [172]:
finalTable = pd.concat([US_Can_data_EN, caseTableSimple], ignore_index=True)
finalTable

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,New York,US,3/24/2020 14:08,23230,183,0.0
1,New Jersey,US,3/24/2020 14:08,2844,27,1.0
2,WA,US,3/24/2020 14:08,2221,111,1.0
3,California,US,3/24/2020 14:08,2212,43,6.0
4,Michigan,US,3/24/2020 14:08,1329,15,0.0
...,...,...,...,...,...,...
257,,St. Vincent Grenadines,3/24/2020 14:08,1,0,0.0
258,,Somalia,3/24/2020 14:08,1,0,0.0
259,,Syria,3/24/2020 14:08,1,0,0.0
260,,Timor-Leste,3/24/2020 14:08,1,0,0.0


In [173]:
timeStampe = currentTime.strftime('%m_%d_%Y_%H_%M')
finalTable.to_csv('./web_data/{}_webData.csv'.format(timeStampe), index=False)

# Scrap data for China

In [27]:
# Test if we can scrap info from worldometers
# The communication with website is ok if the response is 200
CHN = "https://ncov.dxy.cn/ncovh5/view/pneumonia?scene=2&clicktime=1579582238&enterid=1579582238&from=singlemessage&isappinstalled=0"
response3 = get(CHN, headers=headers)
response3.encoding='utf-8' ##去掉这句则乱码，加上则正常显示，其中utf-8是根据网页源代码中设置的编码格式来指定的  
response3

<Response [200]>

In [28]:
# Scrap all content from the website
html_soup3 = BeautifulSoup(response3.text, 'html.parser')

In [76]:
print(html_soup3.prettify())

<!DOCTYPE html>
<html lang="zh-cn" xmlns:layout="http://www.ultraq.net.nz/web/thymeleaf/layout">
 <head>
  <link href="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/umi.bundle.css?t=1583497981741" rel="stylesheet"/>
  <meta charset="utf-8"/>
  <meta content="width=device-width,initial-scale=1,user-scalable=0,viewport-fit=cover" name="viewport"/>
  <meta content="#000000" name="theme-color"/>
  <title>
   全球新冠病毒最新实时疫情地图_丁香园
  </title>
  <script>
   window.routerBase = "/ncovh5/view";
  </script>
  <script charset="utf-8" src="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/vendors~p__ECommerce~p__Pneumonia~p__Pneumonia__area~p__Pneumonia__recommend-list~p__Pneumonia__rumo~5e297593.async.13df3f6e.js">
  </script>
  <script charset="utf-8" src="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/vendors~p__Pneumonia~p__Pneumonia__area~p__Pneumonia__rumor-list.async.9184546f.js">
  </script>
  <link href="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/vendors~p__ECommerce~p__Pneumonia~p__Pneumonia__are

In [80]:
html_soup3.find_all('script', id='getAreaStat')[0].contents

['try { window.getAreaStat = [{"provinceName":"湖北省","provinceShortName":"湖北","currentConfirmedCount":19568,"confirmedCount":67707,"suspectedCount":0,"curedCount":45153,"deadCount":2986,"comment":"","locationId":420000,"statisticsData":"https://file1.dxycdn.com/2020/0223/618/3398299751673487511-135.json","cities":[{"cityName":"武汉","currentConfirmedCount":17634,"confirmedCount":49912,"suspectedCount":0,"curedCount":29908,"deadCount":2370,"locationId":420100},{"cityName":"孝感","currentConfirmedCount":369,"confirmedCount":3518,"suspectedCount":0,"curedCount":3024,"deadCount":125,"locationId":420900},{"cityName":"鄂州","currentConfirmedCount":352,"confirmedCount":1394,"suspectedCount":0,"curedCount":988,"deadCount":54,"locationId":420700},{"cityName":"随州","currentConfirmedCount":187,"confirmedCount":1307,"suspectedCount":0,"curedCount":1077,"deadCount":43,"locationId":421300},{"cityName":"宜昌","currentConfirmedCount":170,"confirmedCount":931,"suspectedCount":0,"curedCount":727,"deadCount":34,"l

In [ ]:
{"provinceName":"湖北省","provinceShortName":"湖北","currentConfirmedCount":19568,"confirmedCount":67707,"suspectedCount":0,"curedCount":45153,"deadCount":2986,"comment":"","locationId":420000,"statisticsData":"https://file1.dxycdn.com/2020/0223/618/3398299751673487511-135.json","cities":[{"cityName":"武汉","currentConfirmedCount":17634,"confirmedCount":49912,"suspectedCount":0,"curedCount":29908,"deadCount":2370,"locationId":420100},{"cityName":"孝感","currentConfirmedCount":369,"confirmedCount":3518,"suspectedCount":0,"curedCount":3024,"deadCount":125,"locationId":420900},{"cityName":"鄂州","currentConfirmedCount":352,"confirmedCount":1394,"suspectedCount":0,"curedCount":988,"deadCount":54,"locationId":420700},{"cityName":"随州","currentConfirmedCount":187,"confirmedCount":1307,"suspectedCount":0,"curedCount":1077,"deadCount":43,"locationId":421300},{"cityName":"宜昌","currentConfirmedCount":170,"confirmedCount":931,"suspectedCount":0,"curedCount":727,"deadCount":34,"locationId":420500},{"cityName":"荆州","currentConfirmedCount":155,"confirmedCount":1580,"suspectedCount":0,"curedCount":1376,"deadCount":49,"locationId":421000},{"cityName":"黄冈","currentConfirmedCount":151,"confirmedCount":2907,"suspectedCount":0,"curedCount":2631,"deadCount":125,"locationId":421100},{"cityName":"荆门","currentConfirmedCount":146,"confirmedCount":928,"suspectedCount":0,"curedCount":743,"deadCount":39,"locationId":420800},{"cityName":"黄石","currentConfirmedCount":95,"confirmedCount":1015,"suspectedCount":0,"curedCount":884,"deadCount":36,"locationId":420200},{"cityName":"十堰","currentConfirmedCount":93,"confirmedCount":672,"suspectedCount":0,"curedCount":571,"deadCount":8,"locationId":420300},{"cityName":"襄阳","currentConfirmedCount":82,"confirmedCount":1175,"suspectedCount":0,"curedCount":1055,"deadCount":38,"locationId":420600},{"cityName":"仙桃","currentConfirmedCount":53,"confirmedCount":575,"suspectedCount":0,"curedCount":501,"deadCount":21,"locationId":429004},{"cityName":"天门","currentConfirmedCount":24,"confirmedCount":496,"suspectedCount":0,"curedCount":457,"deadCount":15,"locationId":429006},{"cityName":"咸宁","currentConfirmedCount":21,"confirmedCount":836,"suspectedCount":0,"curedCount":801,"deadCount":14,"locationId":421200},{"cityName":"潜江","currentConfirmedCount":21,"confirmedCount":198,"suspectedCount":0,"curedCount":168,"deadCount":9,"locationId":429005},{"cityName":"恩施州","currentConfirmedCount":15,"confirmedCount":252,"suspectedCount":0,"curedCount":231,"deadCount":6,"locationId":422800},{"cityName":"神农架林区","currentConfirmedCount":0,"confirmedCount":11,"suspectedCount":0,"curedCount":11,"deadCount":0,"locationId":429021}]}